<a href="https://colab.research.google.com/github/sadhvikreddy/monte-carlo-simulation/blob/main/monte_carlo_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [93]:
import pandas as pd
import random

In [19]:
file = open('sample_input.txt', 'r')

input = pd.read_csv(file, delimiter='\t')

input.head()

,Golfer,Mean,Standard deviation
0,Sergio Garcia,68.998021,2.802211
1,Tiger Woods,69.351350,2.789151
2,Kenny Perry,69.914719,2.779165
3,Gonzalo Fernandez-Castano,70.004543,2.807307
4,Rory McIlroy,70.039146,2.815256


In [79]:
input.columns = input.columns.str.strip()
transformed = pd.DataFrame(columns=['golfer', 'mean', 'standard_deviation'])
transformed['golfer'] = input['Golfer']
transformed['mean'] = pd.to_numeric(input['Mean'])
transformed['standard_deviation'] = pd.to_numeric(input['Standard deviation'])

transformed['position'] = pd.factorize(transformed['mean'])[0] + 1

transformed.head(25)

,golfer,mean,standard_deviation,position
0,Sergio Garcia,68.998021,2.802211,1
1,Tiger Woods,69.351350,2.789151,2
2,Kenny Perry,69.914719,2.779165,3
3,Gonzalo Fernandez-Castano,70.004543,2.807307,4
4,Rory McIlroy,70.039146,2.815256,5
5,John Cook,70.073148,2.748841,6
6,Jaco Van Zyl,70.089262,2.784261,7
7,Brandt Snedeker,70.179877,2.771891,8
8,Francesco Molinari,70.216814,2.748345,9
9,Matteo Manassero,70.220108,2.773465,10


In [228]:
# Generic function for win friction
def calculate_column_friction(column, position, total_in_position):
  return 1 / total_in_position if column <= position else 0

# caluclate win friction based on the requirement
def caluclate_win_frictions(df):
  df['win_friction'] = df['position'].apply(lambda c: calculate_column_friction(c, 1, df['position'].eq(c).sum()))
  df['top_5_friction'] = df['position'].apply(lambda c: calculate_column_friction(c, 5, df['position'].eq(c).sum()))
  return df

In [156]:
# for one tournament simulation, we generate the values for 4 rounds and add them.
#

def generateOneSimulation():
  one_simulation = pd.DataFrame(columns=['golfer', 'generated_score'])

  one_simulation['golfer'] = transformed['golfer']
  one_simulation['generated_score_1'] = random.normalvariate(transformed['mean'], transformed['standard_deviation']).astype(int)
  one_simulation['generated_score_2'] = random.normalvariate(transformed['mean'], transformed['standard_deviation']).astype(int)
  one_simulation['generated_score_3'] = random.normalvariate(transformed['mean'], transformed['standard_deviation']).astype(int)
  one_simulation['generated_score_4'] = random.normalvariate(transformed['mean'], transformed['standard_deviation']).astype(int)
  one_simulation['generated_score'] = one_simulation['generated_score_1'] + one_simulation['generated_score_2'] + one_simulation['generated_score_3'] + one_simulation['generated_score_4']
  one_simulation['position'] = pd.factorize(one_simulation['generated_score'])[0] + 1

  one_simulation_with_fractions = caluclate_win_frictions(one_simulation)
  return one_simulation_with_fractions

In [188]:
def run_simulations(n):
 simulations = []
 for i in range(n):
  oneSim = generateOneSimulation()
  simulations.append({'table': oneSim})


 for_avg = pd.concat([a['table'] for a in simulations])
 averages = (for_avg.groupby('golfer')[['win_friction','top_5_friction']].mean())

 print(averages.sort_values(by=['win_friction'], ascending=False))

In [210]:
run_simulations(10)

                            win_friction  top_5_friction
golfer                                                  
Sergio Garcia                        0.9        0.900000
Tiger Woods                          0.1        0.833333
Brandt Snedeker                      0.0        0.148889
Charl Schwartzel                     0.0        0.072222
Duffy Waldorf                        0.0        0.072222
Darren Fichardt                      0.0        0.165972
Francesco Molinari                   0.0        0.148889
Gonzalo Fernandez-Castano            0.0        0.218472
Jim Furyk                            0.0        0.123889
John Cook                            0.0        0.153889
Jaco Van Zyl                         0.0        0.138889
Jay Haas                             0.0        0.257639
Kenny Perry                          0.0        0.293472
Justin Rose                          0.0        0.086389
Luke Donald                          0.0        0.123889
Matteo Manassero               

In [215]:
run_simulations(100)

                            win_friction  top_5_friction
golfer                                                  
Sergio Garcia                      0.905        0.905000
Tiger Woods                        0.095        0.895000
Brandt Snedeker                    0.000        0.133161
Charl Schwartzel                   0.000        0.119634
Duffy Waldorf                      0.000        0.111301
Darren Fichardt                    0.000        0.110364
Francesco Molinari                 0.000        0.121621
Gonzalo Fernandez-Castano          0.000        0.236647
Jim Furyk                          0.000        0.092133
John Cook                          0.000        0.163679
Jaco Van Zyl                       0.000        0.165703
Jay Haas                           0.000        0.146305
Kenny Perry                        0.000        0.461575
Justin Rose                        0.000        0.094816
Luke Donald                        0.000        0.092002
Matteo Manassero               

In [222]:
run_simulations(1000)

                            win_friction  top_5_friction
golfer                                                  
Sergio Garcia                      0.912        0.912000
Tiger Woods                        0.088        0.891760
Brandt Snedeker                    0.000        0.139345
Charl Schwartzel                   0.000        0.118995
Duffy Waldorf                      0.000        0.115710
Darren Fichardt                    0.000        0.136646
Francesco Molinari                 0.000        0.115648
Gonzalo Fernandez-Castano          0.000        0.231378
Jim Furyk                          0.000        0.090944
John Cook                          0.000        0.177517
Jaco Van Zyl                       0.000        0.166128
Jay Haas                           0.000        0.180489
Kenny Perry                        0.000        0.436085
Justin Rose                        0.000        0.091270
Luke Donald                        0.000        0.091541
Matteo Manassero               

In [223]:
run_simulations(5000)

                            win_friction  top_5_friction
golfer                                                  
Sergio Garcia                     0.9118        0.911800
Tiger Woods                       0.0882        0.888181
Brandt Snedeker                   0.0000        0.146564
Charl Schwartzel                  0.0000        0.117834
Duffy Waldorf                     0.0000        0.112447
Darren Fichardt                   0.0000        0.135746
Francesco Molinari                0.0000        0.116176
Gonzalo Fernandez-Castano         0.0000        0.226963
Jim Furyk                         0.0000        0.093589
John Cook                         0.0000        0.184212
Jaco Van Zyl                      0.0000        0.173091
Jay Haas                          0.0000        0.155819
Kenny Perry                       0.0000        0.446534
Justin Rose                       0.0000        0.094423
Luke Donald                       0.0000        0.093299
Matteo Manassero               

In [224]:
run_simulations(5000)

                            win_friction  top_5_friction
golfer                                                  
Sergio Garcia                   0.912567        0.912567
Tiger Woods                     0.087367        0.894006
Kenny Perry                     0.000067        0.440081
Brandt Snedeker                 0.000000        0.136676
Duffy Waldorf                   0.000000        0.114388
Darren Fichardt                 0.000000        0.136247
Charl Schwartzel                0.000000        0.119306
Francesco Molinari              0.000000        0.117792
Jay Haas                        0.000000        0.162137
Jim Furyk                       0.000000        0.094501
Jaco Van Zyl                    0.000000        0.167456
Gonzalo Fernandez-Castano       0.000000        0.231401
Justin Rose                     0.000000        0.094711
John Cook                       0.000000        0.180910
Luke Donald                     0.000000        0.094166
Matteo Manassero               

In [225]:
run_simulations(5000)

                            win_friction  top_5_friction
golfer                                                  
Sergio Garcia                     0.9137        0.913700
Tiger Woods                       0.0863        0.889132
Brandt Snedeker                   0.0000        0.146581
Charl Schwartzel                  0.0000        0.116777
Duffy Waldorf                     0.0000        0.111465
Darren Fichardt                   0.0000        0.131851
Francesco Molinari                0.0000        0.120359
Gonzalo Fernandez-Castano         0.0000        0.230549
Jim Furyk                         0.0000        0.094358
John Cook                         0.0000        0.185099
Jaco Van Zyl                      0.0000        0.170260
Jay Haas                          0.0000        0.156419
Kenny Perry                       0.0000        0.441093
Justin Rose                       0.0000        0.095201
Luke Donald                       0.0000        0.093344
Matteo Manassero               

In [226]:
run_simulations(5000)

                            win_friction  top_5_friction
golfer                                                  
Sergio Garcia                      0.911        0.911000
Tiger Woods                        0.089        0.889972
Brandt Snedeker                    0.000        0.143497
Charl Schwartzel                   0.000        0.115196
Duffy Waldorf                      0.000        0.111681
Darren Fichardt                    0.000        0.128954
Francesco Molinari                 0.000        0.117180
Gonzalo Fernandez-Castano          0.000        0.229173
Jim Furyk                          0.000        0.093027
John Cook                          0.000        0.182873
Jaco Van Zyl                       0.000        0.166245
Jay Haas                           0.000        0.159639
Kenny Perry                        0.000        0.450725
Justin Rose                        0.000        0.093640
Luke Donald                        0.000        0.092875
Matteo Manassero               

In [227]:
run_simulations(5000)

                            win_friction  top_5_friction
golfer                                                  
Sergio Garcia                     0.9107        0.910700
Tiger Woods                       0.0893        0.891302
Brandt Snedeker                   0.0000        0.135381
Charl Schwartzel                  0.0000        0.119643
Duffy Waldorf                     0.0000        0.112357
Darren Fichardt                   0.0000        0.132610
Francesco Molinari                0.0000        0.117209
Gonzalo Fernandez-Castano         0.0000        0.226301
Jim Furyk                         0.0000        0.092412
John Cook                         0.0000        0.179851
Jaco Van Zyl                      0.0000        0.167776
Jay Haas                          0.0000        0.162001
Kenny Perry                       0.0000        0.438145
Justin Rose                       0.0000        0.093585
Luke Donald                       0.0000        0.093510
Matteo Manassero               

With 5000 iterations, the results look stable.